# Milestone Project 2 - Blackjack Game

## Problem Statement

Create a Complete BlackJack Card Game in Python.

### Requirements

* You need to create a simple text-based [BlackJack](https://en.wikipedia.org/wiki/Blackjack) game
* The game needs to have one player versus an automated dealer.
* The player can stand or hit.
* The player must be able to pick their betting amount.
* You need to keep track of the player's total money.
* You need to alert the player of wins, losses, or busts, etc...

### Implementation Style

* OOP and classes in Python

## Design Process

### Game Card
* A game card is printed with a box dash. The card rank is printed as well
* All dealer's cards are hidden except the first card

### Classes, Methods, global Variables
* Create global variables to track:
  * current player
  * card suit, rank and Value
* Create a Card, Deck, and Bet classes

### Card Values
* All face cards (**Jack (J), Queen (Q), King (K)**) count as 10.
* Ace counts as 1 or 11 according to player choice
  * An ace can turn to 1 when
    * the game winner needs to be determined and a player sum exceeds 21 but has an ace among their cards
      * In this case, if the sum will be less than 21 with ace  becomes 1. 
      * This is a valid operation and they player has therefore not **BUST**
* All other cards count for their number 2 to 10.

### Determine number of decks to play with
  
### Player Options
* A player has the following options:
  * Split
    * A split occurs if the first two cards have the same value.
    * The player has the option to split or perform other actions
    * If a player decides to split
      * The two cards are split into separate bets
      * Each split has its own bet and the player can perform independent actions on each split
    * Note:
      * Multiple split can occur within a split 
  * Stand
    * A player **Stand** means the player has decided to take no more cards.
      * Note:
        * When a player stands, the dealer must reveal their cards.
          * If the dealer cannot take a stand yet, the dealer must continue to hit until the dealer hits a stand
            * This means the sum of dealers can must be at least 17.
  * Hit
    * If a player decides to **HIT**, the player must add an extra card to their cards
      * Note:
        * If a player card sum exceeds 21 after a **HIT**, that player has BUST. 
          * This means the dealer automatically wins the round!
  * Double
    * If a player decides to Double, this means the player doubles their wager
    * Next, the player must **HIT**. 
    * Finally, the game condition will be checked and a winner will be determined.
      * Note:
        * **Dealer Stand** must take effect. 

### Game States
* BlackJack
  * If a player has a total of 21 from the first two cards, that is a **BlackJack**.
    * This means this player has automatically won the round
* Push
  * A push occurs when the dealer and player have the total card sum
  * The player and dealer gets to keep their wager
* Bust
  * A bust occurs when one of the players exceed 21
  * A bust occurs for a player has a higher score than the dealer 
  * Loser loses their bet.
* Win
  * A win occurs when a player has a lower card sum than the dealer
  * A win can also occur if a dealer or player **BUST**
  * The winners get to keep the total wager
  * Note:
    * If both players exceed a total of 21, the player with the lower score gets 
* Dealer Stand
  * A dealer must relieve their hidden cards when a player decided to **STAND**.
  * Note:
    * After the dealer relieves their hidden cards,
    * If the dealer's sum is less than 17
      * the dealer must hit until a sum of at least 17 has been reached 
    * Once this condition is satisfied:
      * Check game condition and determine the winner


### Continue Games
* If a game status has been determined, Ask player if they would like to continue the game?

### Reset game
- Reset deck

In [218]:
# Global Variable
import random
from abc import ABC, abstractmethod

suits = ('spades', 'hearts', 'diamonds', 'clubs')
ranks = ('ace','two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'jack', 'queen', 'king')
values = {'two': 2, 'three': 3,'four': 4, 'five': 5, 'six':6, 'seven':7, 'eight':8, 'nine':9, 'ten':10, 'jack': 10, 'queen': 10, 'king': 10, 'ace': 11}
card_emojis = {'spades': '♤', 'hearts': '♡', 'diamonds': '♢', 'clubs': '♣️'}

In [219]:
class Card:
    # constructor
    def __init__(self, suit, rank) -> None:
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
        self.emoji = card_emojis[suit]

    def cardValue(self) -> str:
        return self.value

    def printCard(self) -> None:
        print("*"*15)
        if len(str(self.value)) == 1:
            print("* {value}           *".format(value=self.value))
        else:
            print("* {value}          *".format(value=self.value))
        print("* {emoji}           *".format(emoji=self.emoji))
        print("*             *")
        print("*      {emoji}      *".format(emoji=self.emoji))
        print("*             *")
        print("*           {emoji} *".format(emoji=self.emoji))
        if len(str(self.value)) == 1:
            print("*           {value} *".format(value=self.value))
        else:
            print("*          {value} *".format(value=self.value))
        print("*"*15)
    def __str__(self) -> str:
        return '{} of {}'.format(self.rank, self.suit)    

In [220]:
card = Card('spades', 'ace')
card1 = Card('clubs', 'two')
card.printCard()
card1.printCard()

***************
* 11          *
* ♤           *
*             *
*      ♤      *
*             *
*           ♤ *
*          11 *
***************
***************
* 2           *
* ♣️           *
*             *
*      ♣️      *
*             *
*           ♣️ *
*           2 *
***************


In [221]:
# Deck Class
class Deck:
    # constructor
    def __init__(self, num_cards = 1):
        # Populate each deck with all cards
        self.cards = []
        for suit in suits:
            for rank in ranks:
                self.cards.append(Card(suit, rank))
        # double the size of the deck based on the number of cards provided
        for num in range(1, num_cards):
            self.cards += self.cards

    # Functions
    # shuffle a deck
    def shuffle_deck(self) -> None:
        random.shuffle(self.cards)
    # Deal one card from the deck
    def deal_card(self) -> str:
        return self.cards.pop()
    # Remove a card from deck

    # Helper Methods
    def __len__(self) -> int:
        return len(self.cards)
    def __str__(self) -> str:
        return "The deck has {total} cards".format(total=len(self))

In [222]:
# Create Chip | Bet class
class Chip:
    def __init__(self, amount=200) -> None:
        self.amount = amount
        self.total = amount
        # track how much the player has lost or won
        self.totalWon = 0
        self.totalBet = 0

    def initialAmount(self) -> int:
        return self.amount

    def balance(self) -> int:
        return self.total

    def withdraw(self, amount) -> None:
        # Edge case: you can only bet if amount is <= total balance
        if(amount > self.total):
            print('Insufficient balance! Request amount ${amount} exceeds current balance ${balance}!'.format(amount=amount, balance = self.total))
        else:
            self.total -= amount
            self.totalBet += amount
            print('Successfully deposited ${amount}. New balance is ${balance}!'.format(amount=amount, balance = self.total))

    def deposit(self, amount) -> None:
        if(amount < 0):
            print('Failed! Only positive transactions are allowed')
        else:
            # update total balance and the totalWon
            self.total += amount
            self.totalWon += amount
            print('Successfully credited ${amount}. New balance is ${balance}!'.format(amount=amount, balance = self.total))
            
    def getTotalBet(self) -> int:
        return self.totalBet

    def getTotalWon(self) -> int:
        return self.totalWon

    def __str__(self) -> str:
        return "Total: ${total}.\nTotal won: ${total_won}.\nTotal bet: ${total_bet}".format(total=self.total, total_won=self.totalWon, total_bet=self.totalBet)


In [223]:
# Create an abstract class for all types of players
class Actor(ABC):
    # constructor
    # create a container for player cards, playerCardTotal, 
    def __init__(self, name, amount) -> None:
        self.name = name
        self.chips = Chip(amount=amount)
        self.cards = []
        self.cardsTotal = 0
        # create a status to track which player has won
    
    # abstract functions
    @abstractmethod
    def place_bet(self, amount) -> None:
        pass

    @abstractmethod
    def credit_bet(self, amount) -> None:
        pass

    @abstractmethod
    def add_card(self, card) -> None:
        pass

    # member functions
    @property
    def player_name(self) -> str:
        return self.name
    
    @property
    def getCardsTotal(self) -> int:
        return self.cardsTotal
    
    def __str__(self):
        return "{name}'s stats.\nInitial AmountCard total: {card_total}.\nCards: {cards}.\nTotal balance: {balance}".format(name=self.name, card_total=self.cardsTotal,cards=self.cards,balance=self.chips.balance() )
    

In [224]:
class Dealer(Actor):
    # constructor
    def __init__(self, name, amount=1000):
        super().__init__(name, amount)
    
    # abstract functions
    def place_bet(self, amount) -> None:
        print(self.cardsTotal)

    def credit_bet(self, amount) -> None:
        print(self.cardsTotal)

    def add_card(self, card) -> None:
        print(self.cardsTotal)

    def dealer_stand() -> None:
        pass

In [225]:
class Player(Actor):
    # constructor
    def __init__(self,name, amount=200):
        super().__init__(name, amount)
    
    # abstract functions
    def place_bet(self, amount) -> None:
        print(self.cardsTotal)

    def credit_bet(self, amount) -> None:
        print(self.cardsTotal)

    def add_card(self, card) -> None:
        print(self.cardsTotal)

    # member functions
    def bust() -> None:
        pass

    def split() -> None:
        pass

    def double() -> None:
        pass

    def hit() -> None:
        pass

In [226]:
dealer = Dealer(name='dealer')
player = Player(name='player1')
print(dealer)
dealer.place_bet(50)
dealer.credit_bet(50)
dealer.add_card(50)
print(player)
player.place_bet(50)
player.credit_bet(50)
player.add_card(50)


dealer's stats.
Card total: 0.
Cards: [].
Total balance: 1000
0
0
0
player1's stats.
Card total: 0.
Cards: [].
Total balance: 200
0
0
0


In [227]:
# functions
def welcome_message():
    pass

# display score board

# display players current count

In [228]:
# Game setup